In [1]:
import os, re
os.environ['PYTHONPATH'] = '/home/aksha/Workbench/Research/Labs/e-lab/parser/constrain'
from pydantic import BaseModel, Field
from dataclasses import dataclass
from typing import List, Union, Optional, Type


In [2]:
from constrain.grammars.gnbf import GNBF

In [3]:
class ThoughtState(BaseModel):
    thought: str 
    goal: str 
    tool: str = Field(..., description="Choose one of ['Web_QA', 'Web_Search', 'Web_Scraping', 'Web_Automation', 'Web_Research']")
    action: str = Field(..., description="Choose one of ['Create', 'Update', 'Delete', 'Read']")
    action_input: str = Field(..., description="The input data for the action")
    thought_id: Optional[str] = Field(None, description="The unique identifier for the thought")
    

print(ThoughtState.schema())
print('-------------------')
converter = GNBF(ThoughtState.schema())
grammar = converter.generate_grammar()
converter.verify_grammar(grammar)

{'title': 'ThoughtState', 'type': 'object', 'properties': {'thought': {'title': 'Thought', 'type': 'string'}, 'goal': {'title': 'Goal', 'type': 'string'}, 'tool': {'title': 'Tool', 'description': "Choose one of ['Web_QA', 'Web_Search', 'Web_Scraping', 'Web_Automation', 'Web_Research']", 'type': 'string'}, 'action': {'title': 'Action', 'description': "Choose one of ['Create', 'Update', 'Delete', 'Read']", 'type': 'string'}, 'action_input': {'title': 'Action Input', 'description': 'The input data for the action', 'type': 'string'}, 'thought_id': {'title': 'Thought Id', 'description': 'The unique identifier for the thought', 'type': 'string'}}, 'required': ['thought', 'goal', 'tool', 'action', 'action_input']}
-------------------


from_string grammar:
root ::= thoughtstate ws 
thoughtstate ::= [{] ws ["] [t] [h] [o] [u] [g] [h] [t] ["] [:] ws string [,] ws ["] [g] [o] [a] [l] ["] [:] ws string [,] ws ["] [t] [o] [o] [l] ["] [:] ws string [,] ws ["] [a] [c] [t] [i] [o] [n] ["] [:] ws string [,] ws ["] [a] [c] [t] [i] [o] [n] [-] [i] [n] [p] [u] [t] ["] [:] ws string [,] ws ["] [t] [h] [o] [u] [g] [h] [t] [-] [i] [d] ["] [:] ws string [}] ws 
ws ::= ws_4 
string ::= ["] string_7 ["] ws 
ws_4 ::= [ <U+0009><U+000A>] ws_4 | 
string_5 ::= [^"\] | [\] string_6 
string_6 ::= ["\/bfnrt] | [u] [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f] 
string_7 ::= string_5 string_7 | 



from_string grammar:
root ::= thoughtstate ws 
thoughtstate ::= [{] ws ["] [t] [h] [o] [u] [g] [h] [t] ["] [:] ws string [,] ws ["] [g] [o] [a] [l] ["] [:] ws string [,] ws ["] [t] [o] [o] [l] ["] [:] ws string [,] ws ["] [a] [c] [t] [i] [o] [n] ["] [:] ws string [,] ws ["] [a] [c] [t] [i] [o] [n] [-] [i] [n] [p] [u] [t] ["] [:] ws string [,] ws ["] [t] [h] [o] [u] [g] [h] [t] [-] [i] [d] ["] [:] ws string [}] ws 
ws ::= ws_4 
string ::= ["] string_7 ["] ws 
ws_4 ::= [ <U+0009><U+000A>] ws_4 | 
string_5 ::= [^"\] | [\] string_6 
string_6 ::= ["\/bfnrt] | [u] [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f] 
string_7 ::= string_5 string_7 | 



In [471]:
print(grammar)

root ::= thoughtstate ws
thoughtstate ::= "{" ws "\"thought\":" ws string "," ws "\"goal\":" ws string "," ws "\"tool\":" ws string "," ws "\"action\":" ws string "," ws "\"action-input\":" ws string "," ws "\"thought-id\":" ws string "}" ws
ws ::= [ \t\n]*
string ::= 
            "\"" (
            [^"\\] |
            "\\" (["\\/bfnrt] | "u" [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f] [0-9a-fa-f])
            )* "\"" ws
        


In [472]:
from transformers import AutoTokenizer
import transformers
import torch

model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [473]:
text = """ 
```
{
"ThoughtState": {
"thought": "Vladimir Putin is the current President of Russia.",
"goal": "To provide information about Vladimir Putin.",
"tool": "Web_Search",
"action": "Read",
"action_input": "Vladimir Putin biography",
"thought_id": "12345"
    }
}
```
""" 

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium", padding_side='left')
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# source: https://huggingface.co/microsoft/DialoGPT-medium

# encode the new user input, add the eos_token and return a tensor in Pytorch
new_user_input_ids = tokenizer.encode("Can you generate a simple JSON object?" + tokenizer.eos_token, return_tensors='pt')

# append the new user input tokens to the chat history
bot_input_ids = new_user_input_ids

# generated a response while limiting the total chat history to 1000 tokens, 
chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

# pretty print last output tokens from bot
print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


DialoGPT: I'm not sure what you mean by that.


In [474]:
terminal_tokens_ids = [tokenizer.encode(token)[0] for token in ['{', '}', '"', ':', ',']]

In [475]:
content_boost_factor = 1.1  # Slightly boost content token probabilities
structure_boost_factor = 5  # Significantly boost terminal token probabilities

In [2]:
%pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 3.7 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-v0.1", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-v0.1")

prompt = "My favourite condiment is"

model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
model.to(device)

generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
tokenizer.batch_decode(generated_ids)[0]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [361]:
json_string = r'{"key": "value", "array": [1, 2, 3], "bro": {"ayo": "huh"}, "nested": {"another_key": "another_value"}} ohhhh'
JSON.parse(json_string)

{'key': 'value',
 'array': [1, 2, 3],
 'bro': {'ayo': 'huh'},
 'nested': {'another_key': 'another_value'}}

In [363]:
toml_string = '''
[products]
name = "Hammer"
sku = 738594937

[products]

[products]
name = "Nail"
sku = 284758393
color = "gray"

[products.sub]
hey = "yo"
'''
print(TOML.parse(toml_string))

{'products': [{'name': 'Hammer', 'sku': 738594937}, {}, {'name': 'Nail', 'sku': 284758393, 'color': 'gray'}], 'products.sub': [{'hey': 'yo'}]}


In [365]:
text = """ 
<studentsList>
    <student id="1">
        <firstName>Greg</firstName>
        <lastName>Dean</lastName>
        <certificate>True</certificate>
        <scores>
            <module1>70</module1>
            <module12>80</module12>
            <module3>90</module3>
        </scores>
    </student>
    <student ind="2">
        <firstName>Wirt</firstName>
        <lastName>Wood</lastName>
        <certificate>True</certificate>
    </student>
</studentsList>
"""

text = """ 
<studentsList>
    <student ind="2">
        <firstName>Wirt</firstName>
        <lastName>Wood</lastName>
        <certificate>True</certificate>
    </student>
</studentsList>
"""

print(json.dumps(parse_xml(text), indent=4))

{
    "studentsList": {
        "student": {
            "attributes": {
                "ind": "2"
            },
            "firstName": {
                "value": "Wirt"
            },
            "lastName": {
                "value": "Wood"
            },
            "certificate": {
                "value": "True"
            }
        }
    }
}


In [366]:
class Complex(BaseModel):
    real: float
    imaginary: float

print(XML.make_format([{'model': Complex}], 'single')[0])
print(TOML.make_format([{'model': Complex}], 'single')[0])
print(JSON.make_format([{'model': Complex}], 'single')[0])

Complex:
```
<Complex>
<real> #float# </real>
<imaginary> #float# </imaginary>
</Complex>
```


Complex:
```
[Complex]
real = # Type: float
imaginary = # Type: float
```



```
{
"Complex": {
real = # Type: float
imaginary = # Type: float
    }
}
```




In [443]:
class TOML:
    @staticmethod
    def make_format(tasks: List[dict], return_sequence: str) -> str:
        grammar, instruct = "", []
        for task in tasks:
            model = task.get('model')
            command = task.get('task_name', '')
            if isinstance(model, list):
                name = "_".join([m.__name__ for m in model])
            else:
                name = model.__name__
                model = [model]
            instruct.append(name)

            variables = ModelParser.extract_variables_with_descriptions(model)
            forma = TOML.generate_prompt_from_variables(variables, nested=True)
            grammar += f"{name}:\n```\n{forma}\n```\n\n"

        return grammar, instruct

    @staticmethod
    def generate_prompt_from_variables(variables_info: dict, nested: bool = False) -> str:
        prompt_lines = []
        for model_name, fields in variables_info.items():
            if nested:
                prompt_lines.append(f"[{model_name}]")
            for var_name, details in fields.items():
                line = f'{var_name} = '
                if details.get('description'):
                    line += f'"{details["description"]}"'
                line += f'# Type: {details["type"]}'
                if str(details.get("default")) not in ['PydanticUndefined', 'None']:
                    line += f', Default: "{details["default"]}"'
                prompt_lines.append(line)
        return "\n".join(prompt_lines)

    @staticmethod
    def _generate_single_model_prompt(fields: dict, model_name: str, nested: bool = False) -> dict:
        model_data = {}
        for var_name, details in fields.items():
            model_data[var_name] = {
                'description': details['description'],
                'type': details['type'],
                'default': details['default'] if str(details.get("default")) != 'PydanticUndefined' else None
            }
        return model_data
    
    @staticmethod
    def parse_toml(toml_string):
        def parse_section(toml_string, i):
            start = i
            while toml_string[i] != ']':
                i += 1
            key = toml_string[start:i]
            i = skip_whitespace(toml_string, i + 1)
            section = {key: {}}
            while i < len(toml_string) and toml_string[i] not in '[':
                subkey, i = parse_key(toml_string, i)
                i = skip_whitespace(toml_string, i)
                if toml_string[i] == '=':
                    i = skip_whitespace(toml_string, i + 1)
                    value, i = parse_value(toml_string, i)
                    section[key][subkey.replace('\n', '')] = value
                i = skip_whitespace(toml_string, i)
            return section, i

        def parse_key(toml_string, i):
            start = i
            while toml_string[i] not in '=':
                i += 1
            return toml_string[start:i], i

        def parse_value(toml_string, i):
            if toml_string[i] == '"':
                print('starting string')
                return parse_string(toml_string, i + 1)
            elif toml_string[i] == '[':
                print('starting array')
                return parse_array(toml_string, i)
            else:
                return parse_number(toml_string, i)

        def parse_string(toml_string, i):
            start = i
            while toml_string[i] != '"':
                print('char', toml_string[i])
                i += 1
            print('string', toml_string[start:i])
            return toml_string[start:i], i 

        def parse_number(toml_string, i):
            start = i
            while toml_string[i] in '0123456789.-':
                i += 1

            val = toml_string[start:i]
            try: 
                return int(val), i
            except ValueError:
                return '', i

        def parse_array(toml_string, i):
            array = []
            i = skip_whitespace(toml_string, i + 1)
            while toml_string[i] != ']':
                print('array', toml_string[:i])
                value, i = parse_value(toml_string, i)
                array.append(value)
                i = skip_whitespace(toml_string, i + 1)
            array = [x for x in array if x]
            return array, i + 1

        def skip_whitespace(toml_string, i):
            while i < len(toml_string) and toml_string[i] in ' \t\n\r':
                print('skipped IN CONSIDERED', toml_string[i])
                i += 1
            print('skipped', toml_string[:i])    
            return i
        
        i = 0
        storage = {}
        while i < len(toml_string):
            i = skip_whitespace(toml_string, i)
            if i < len(toml_string) and toml_string[i] == '[':
                section, i = parse_section(toml_string, i + 1)
                key = list(section.keys())[0]
                if key in storage:
                    storage[key].append(section[key])
                else:
                    storage[key] = [section[key]]
            else:
                break
        return storage
    
    @staticmethod
    def parse(text):
        return TOML.parse_toml(text)

In [444]:
txt = """ 
Man
[BudgetPlan]
total_cost = 500
all_items = ["cake", "balloons", "roses", "ice cream"]
[EventSchedule]
start_time = "12:00 PM"
end_time = "4:00 PM"
activities = ["cake cutting", "balloon decoration", "rose gifting", "ice cream party"]
"""

print(TOML.parse(txt))

skipped IN CONSIDERED  
skipped IN CONSIDERED 

skipped  

{}


In [457]:
from transformers import AutoTokenizer

def tokenize_and_count_hf(text, model_name='bert-base-uncased'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokens = tokenizer.tokenize(text)
    return len(tokens)

# Example usage:
# tokens, count = tokenize_and_count_hf(, 'Hello, world!')
# print(f'Tokens: {tokens}, Count: {count}')


In [11]:
import tiktoken

def tokenize_and_count_tiktoken(text, model_name="gpt-3.5-turbo"):
    encoding = tiktoken.encoding_for_model(model_name)
    tokens = encoding.encode(text)
    return tokens, len(tokens), encoding

In [29]:
tokens = """ 
```
{
"ThoughtState": {
"thought": "Vladimir Putin is the current President of Russia.",
"goal": "To provide information about Vladimir Putin.",
"tool": "Web_Search",
"action": "Read",
"action_input": "Vladimir Putin biography",
"thought_id": "12345"
    }
}
```
"""

expected_tokens = """ 
{
"ThoughtState": {
"thought": "",
"goal": "",
"tool": "",
"action": "",
"action_input": "",
"thought_id": ""
    }
}
"""

In [37]:
class Decoder: 
    def __init__(self, expected): 
        self.encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
        self.expected_tokens  = self.tokenize(expected)
    
    def tokenize(self, text):
        return self.encoding.encode(text)
        
    def get_next_states(current_state, tokens, expected_tokens):
    # Initialize probabilities for each token in the input string
    token_probabilities = [0] * len(tokens)  # Placeholder for actual probability calculation

    # Determine the type of the next expected token (terminal or non-terminal)
    next_expected_token_type = get_token_type(expected_tokens, current_state)

    # Calculate probabilities for each token in the input string
    for i, token in enumerate(tokens):
        token_type = 'terminal' if token in expected_tokens else 'non-terminal'
        if token_type == next_expected_token_type:
            # Assign higher probability if the token matches the expected type
            token_probabilities[i] += calculate_probability(token, i, tokens, expected_tokens)

    # Choose the most probable next token
    next_token_index = token_probabilities.index(max(token_probabilities))
    next_token = tokens[next_token_index]

    return next_token
    
    def get_next_states(self, current_state, tokens):
        token_probabilities = [self.calculate_probability(token, i, tokens) for i, token in enumerate(tokens)]
        # Choose the token with the highest probability score as the most probable next state
        next_token_index = token_probabilities.index(max(token_probabilities))
        next_token = tokens[next_token_index]
        return next_token, next_token_index

    def decode(self, input_string):
        tokens = self.tokenize(input_string)
        path = []
        current_state = 'root'  # Starting state
        token_index = 0

        while token_index < len(tokens):
            current_token = tokens[token_index]
            next_token, next_token_index = self.get_next_states(current_state, tokens[token_index:])
            path.append(next_token)
            token_index += next_token_index + 1  # Move to the index of the next token

            # Update current_state based on your state transition logic
            # Placeholder: current_state = next_token
            current_state = next_token

            if current_state == 'complete':
                break  # Terminate if the decoding process is complete

        return ' '.join(path)

    def calculate_probability(token, index, tokens, expected_tokens):
        """
        Calculates a probability score for a token based on its match with expected type,
        its closeness to the expected position, and its fit within the expected sequence.
        
        :param token: The current token being evaluated.
        :param index: The index of the current token in the input tokens list.
        :param tokens: The list of all tokens in the input string.
        :param expected_tokens: The list of tokens in the expected format, including placeholders for non-terminal tokens.
        :return: A probability score for the token.
        """
        
        # Constants for weighting different factors
        TYPE_WEIGHT = 0.5
        POSITION_WEIGHT = 0.3
        SEQUENCE_WEIGHT = 0.2

        # Determine if the token is terminal or non-terminal
        is_terminal = token in expected_tokens
        type_score = TYPE_WEIGHT if is_terminal else 0

        # Calculate positional closeness score
        # Assuming expected position is based on the order in expected_tokens
        if token in expected_tokens:
            expected_index = expected_tokens.index(token)
            position_diff = abs(index - expected_index)
        else:
            # If it's a non-terminal, we find the closest terminal token's expected position
            closest_terminal_index = min([i for i, t in enumerate(tokens) if t in expected_tokens], key=lambda x: abs(x - index))
            expected_index = expected_tokens.index(tokens[closest_terminal_index])
            position_diff = abs(index - expected_index)
        
        # Normalize position_diff to a score between 0 and 1, assuming max diff is the length of tokens
        max_diff = len(tokens)
        positional_score = POSITION_WEIGHT * (1 - (position_diff / max_diff))

        # Calculate sequence fit score
        # This simplistic approach checks if the next expected token matches the next actual token
        # More complex logic could involve checking subsequences
        if index + 1 < len(tokens) and tokens[index + 1] in expected_tokens:
            next_expected_token = expected_tokens[expected_tokens.index(token) + 1] if token in expected_tokens else None
            sequence_fit_score = SEQUENCE_WEIGHT if tokens[index + 1] == next_expected_token else 0
        else:
            sequence_fit_score = 0

        # Combine scores for final probability
        probability_score = type_score + positional_score + sequence_fit_score
        return probability_score

    def get_token_type(expected_tokens, current_state):
        if current_state in expected_tokens:
            return 'terminal'
        else:
            return 'non-terminal'

In [38]:
decoder = Decoder(expected_tokens)
decoder.decode(tokens)

TypeError: 'int' object is not iterable

In [32]:
expected = tokenize_and_count_tiktoken(expected_tokens)
real = tokenize_and_count_tiktoken(tokens)

In [33]:
get_next_states(0, real, expected)

72

In [35]:
real[2].decode([72])

'i'

In [36]:
real[0]

[720,
 14196,
 4077,
 517,
 1,
 85269,
 1423,
 794,
 341,
 1,
 61665,
 794,
 330,
 53,
 18599,
 31204,
 21810,
 374,
 279,
 1510,
 4900,
 315,
 8524,
 10560,
 1,
 35039,
 794,
 330,
 1271,
 3493,
 2038,
 922,
 36011,
 21810,
 10560,
 1,
 14506,
 794,
 330,
 6109,
 67013,
 761,
 1,
 1335,
 794,
 330,
 4518,
 761,
 1,
 1335,
 6022,
 794,
 330,
 53,
 18599,
 31204,
 21810,
 48345,
 761,
 1,
 61665,
 851,
 794,
 330,
 4513,
 1774,
 702,
 262,
 457,
 534,
 14196,
 4077]